This encompasses the approach to text processing with a more OCR approach.
1. Gray-scale ->  Blur image -> draw ROI of blurred dark area -> Retrieve ROI as Rect -> Extract text using Rect with PyMuPDF
2. Possible drawback:
- Inability to detect text in 

In [1]:
import pymupdf
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

import re
import os
import json
from collections import defaultdict
import difflib
import threading
import collections

In [2]:
import pandas as pd
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from typing import List
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import numpy as np
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import heapq
class MedianFinder(object):
    def __init__(self):
        self.minheap = []
        self.maxheap = []

    def addNum(self, num:float):
        if not self.minheap and not self.maxheap:
            heapq.heappush(self.maxheap, -num)
        else:
            if len(self.minheap)== len(self.maxheap):
                if -self.maxheap[0]<=num:
                    heapq.heappush(self.minheap, num)
                    heapq.heappush(self.maxheap, -heapq.heappop(self.minheap))
                else:
                    heapq.heappush(self.maxheap, -num)
            else:
                if -self.maxheap[0]<= num:
                    heapq.heappush(self.minheap, num)
                else:
                    heapq.heappush(self.minheap, -heapq.heappop(self.maxheap))
                    heapq.heappush(self.maxheap, -num)
                
    def findMedian(self):
        if len(self.minheap)==len(self.maxheap):
            return (self.minheap[0]-self.maxheap[0])/2.0
        else:
            return -self.maxheap[0]

In [4]:

class Node:
    def __init__(self, content):
        self.content = content
        self.children = []
    def add_child(self, child):
        self.children.append(child)
    def __str__(self):
        return f"Node({self.content}, {self.children})"

In [5]:
def should_merge(
    rect1: pymupdf.Rect,
    rect2: pymupdf.Rect,
    h_threshold: float = 4,
    v_threshold: float = 1,
    same_line_height_factor: float = 0.75, 
    min_h_overlap_factor: float = 0.75   # Factor for 'significant' horizontal overlap
) -> bool:
    """
    Determine if two rectangles should be merged based on proximity or overlap.

    Args:
        rect1, rect2: PyMuPDF Rect objects to compare.
        h_threshold: Maximum horizontal gap allowed for merging adjacent rectangles
                     on the same line.
        v_threshold: Maximum vertical gap allowed for merging rectangles that
                     have significant horizontal overlap.
        same_line_height_factor: Multiplier for average height to determine if
                                 rectangles are on the same line (based on y0 diff).
        min_h_overlap_factor: Minimum horizontal overlap (as a fraction of the *smaller* width) required to consider vertical merging.

    Returns:
        bool: True if rectangles should be merged, False otherwise.
    """
    # --- Basic Properties and Checks ---
    if not rect1 or not rect2 or rect1.is_empty or rect2.is_empty:
        return False # Cannot merge with invalid or empty rectangles

    height1 = rect1.height
    height2 = rect2.height
    # Avoid division by zero if heights are zero
    avg_height = (height1 + height2) / 2.0 if (height1 + height2) > 0 else 0

    # --- Condition 1: Horizontal Proximity on Same Line ---
    # Check if vertically aligned enough to be on the same 'line'
    # Use y-centers for potentially better alignment check than just y0
    # is_same_line = abs(rect1.y0 - rect2.y0) < avg_height * same_line_height_factor
    y_center1 = rect1.y0 + height1 / 2.0
    y_center2 = rect2.y0 + height2 / 2.0
    # Rects are on same line if vertical distance between centers is less than avg height
    is_same_line = abs(y_center1 - y_center2) < avg_height if avg_height > 0 else (rect1.y0 == rect2.y0)


    # Calculate horizontal gap (only if they don't overlap horizontally)
    h_gap = -1.0
    if rect1.x1 <= rect2.x0: # rect1 is left of rect2
        h_gap = rect2.x0 - rect1.x1
    elif rect2.x1 <= rect1.x0: # rect2 is left of rect1
        h_gap = rect1.x0 - rect2.x1

    if is_same_line and h_gap >= 0 and h_gap < h_threshold:
        # print(f"H-Merge: {rect1} & {rect2} (gap {h_gap:.2f})") # Debug
        return True # Merge if close horizontally on the same line

    # --- Condition 2: Vertical Proximity with Significant Horizontal Overlap ---
    # Calculate horizontal overlap width
    h_overlap_width = max(0.0, min(rect1.x1, rect2.x1) - max(rect1.x0, rect2.x0))

    # Check if horizontal overlap is significant
    min_width = min(rect1.width, rect2.width)
    has_significant_h_overlap = False
    if min_width > 0 and h_overlap_width / min_width >= min_h_overlap_factor:
        has_significant_h_overlap = True
    elif h_overlap_width > 0 and min_width <= 0: # Overlap exists, one rect has no width? Consider overlap significant.
        has_significant_h_overlap = True
    # Alternative check: One rect is contained horizontally within the other
    is_contained_horizontally = (rect1.x0 >= rect2.x0 and rect1.x1 <= rect2.x1) or \
                                (rect2.x0 >= rect1.x0 and rect2.x1 <= rect1.x1)

    if has_significant_h_overlap or is_contained_horizontally:
        # Calculate vertical gap (only if they don't overlap vertically)
        v_gap = -1.0
        if rect1.y1 <= rect2.y0: # rect1 is above rect2
            v_gap = rect2.y0 - rect1.y1
        elif rect2.y1 <= rect1.y0: # rect2 is above rect1
            v_gap = rect1.y0 - rect2.y1

        if v_gap >= 0 and v_gap < v_threshold:
            # print(f"V-Merge: {rect1} & {rect2} (gap {v_gap:.2f})") # Debug
            return True # Merge if close vertically and overlap significantly horizontally

    # --- Condition 3: Direct Overlap (Optional but good fallback) ---
    # Check if rectangles intersect (overlap) significantly in both dimensions?
    # The previous checks cover proximity *near* overlap.
    # If they actually overlap, should they always merge? Often yes.
    # Calculate vertical overlap
    v_overlap_height = max(0.0, min(rect1.y1, rect2.y1) - max(rect1.y0, rect2.y0))

    # If they overlap both horizontally and vertically, merge them.
    if h_overlap_width > 0 and v_overlap_height > 0:
        # print(f"O-Merge: {rect1} & {rect2}") # Debug
        return True

    return False # No merge conditions met
def merge_text_regions(regions, iterations=3):
    """
    Merge text regions that are close to each other.
    
    Args:
        regions: List of PyMuPDF Rect objects
        iterations: Number of merging passes to perform
    Returns:
        list: Merged PyMuPDF Rect objects
    """
    if not regions:
        return []
    
    # Perform multiple iterations of merging to handle chains of regions
    for _ in range(iterations):
        merged = False
        i = 0
        while i < len(regions):
            j = i + 1
            while j < len(regions):
                if should_merge(regions[i], regions[j], h_threshold=6, v_threshold=1): # Make this more relative
                    # Merge rectangles
                    merged_rect = pymupdf.Rect(
                        min(regions[i].x0, regions[j].x0),
                        min(regions[i].y0, regions[j].y0),
                        max(regions[i].x1, regions[j].x1),
                        max(regions[i].y1, regions[j].y1)
                    )
                    regions[i] = merged_rect
                    regions.pop(j)
                    merged = True
                else:
                    j += 1
            i += 1
            
        if not merged:
            break
            
    return regions

In [6]:
def extract_text_with_ocr_approach(pdf_path, page_num=0, visual_proof = False, is_multi_column = False):
    """
    Extract text from a PDF using image processing to identify text regions.
    With text region merging to combine related text blocks.
    
    Args:
        pdf_path (str): Path to the PDF file
        page_num (int): Page number to process (0-indexed)
        
    Returns:
        list: Extracted text segments from identified regions
    """
    # Open the PDF
    doc = pymupdf.open(pdf_path)
    page = doc[page_num]
    zoom_factor = 2
    # Convert page to image
    pix = page.get_pixmap(matrix=pymupdf.Matrix(zoom_factor, zoom_factor))
    img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.h, pix.w, pix.n)
    
    # Convert to grayscale
    if img.shape[2] >= 3:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    else:
        gray = img[:, :, 0]
    # Apply Gaussian blur
    adaptive = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 13, 2)
    rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (8, 8))
    
    dilated = cv2.dilate(adaptive, rectKernel, iterations=1)
    
    closing = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, rectKernel)
    
    edged = cv2.Canny(closing, 50, 200, apertureSize=3)

    contours, _  = cv2.findContours(
        edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
    
    # Create ROIs from contours
    initial_regions = []
    for cnt in contours:
        
        x, y, w, h = cv2.boundingRect(cnt)
        # Convert back to PDF coordinates (accounting for the zoom factor)
        pdf_x, pdf_y = x/zoom_factor, y/zoom_factor
        pdf_w, pdf_h = w/zoom_factor, h/zoom_factor
        
        # Create PyMuPDF rect
        rect = pymupdf.Rect(pdf_x, pdf_y, pdf_x + pdf_w, pdf_y + pdf_h)
        initial_regions.append(rect)
    
    # Merge text regions that are close to each other
    merged_regions = merge_text_regions(initial_regions, iterations=6)
    merged_regions = [r for r in merged_regions if (r.x1 - r.x0) > 8 and (r.y1 - r.y0) > 8]  
    
    text_regions = []
    
    words = page.get_text("words")
    
    for rect in sorted(merged_regions, key=lambda r: [(r.y1- r.y0) * (r.x1 -r.x0), r.y0, r.x0]):
        text_height_collection = MedianFinder()
        text = ""
        idx = 0
        min_text_height = 1000000
        # Add relative padding to the rect to ensure we catch words that are nearby
        padding = 1.5  # pixels in PDF coordinate space
        rect = pymupdf.Rect(
            rect.x0 - padding,
            rect.y0 - padding,
            rect.x1 + padding,
            rect.y1 + padding
        )
        # Iterate through words and pick out words that fall within the rect
        prev_y = words[0][1] if words else 0
        while words and idx < len(words):
            current_word = words[idx]
            word_rect = pymupdf.Rect(current_word[:4])
            
            if word_rect.intersects(rect) or rect.contains(word_rect):        
                text_height_collection.addNum(word_rect.height)
                min_text_height = min(min_text_height, word_rect.height) 
                text += current_word[4] + (" " if abs(prev_y - current_word[1]) <= abs(current_word[1] - current_word[3])*0.6 else "\n")
                prev_y = current_word[1]
                words.pop(idx)
            else:
                idx += 1
        
        if text.strip():
            if text.strip().isdigit():
                continue
            text_regions.append({"occupy_space": rect,
                                 "content": text, 
                                 "text_height_median": text_height_collection.findMedian(), 
                                 "min_height": min_text_height, 
                                 "page_num": page_num})
    # print(words)
    if visual_proof:
        viz_img = img.copy()
        for item in merged_regions:
            rect = item["occupy_space"]
            # Convert back to image coordinates
            x0, y0, x1, y1 = rect.x0*zoom_factor, rect.y0*zoom_factor, rect.x1*zoom_factor, rect.y1*zoom_factor
            cv2.rectangle(viz_img, (int(x0), int(y0)), (int(x1), int(y1)), (0, 255, 0), 2)
        
        plt.figure(figsize=(15, 15))
        plt.subplot(1, 2, 1)
        plt.imshow(edged, cmap='gray')
        plt.title("Processed Image")
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.imshow(viz_img)
        plt.title(f"Merged Text Regions: {len(text_regions)} blocks")
        plt.axis('off')
        plt.show()
    
    return text_regions[::-1]


In [7]:
from typing import Any, Dict, List
def extract_page_text_via_image_regions(
    page: pymupdf.Page,
    zoom_factor: int = 2,
    adaptive_thresh_block_size: int = 13, # Must be odd > 1
    adaptive_thresh_c: int = 2,
    morph_kernel_size: tuple = (8, 8),
    dilate_iterations: int = 1,
    canny_thresh1: int = 50,
    canny_thresh2: int = 200,
    min_region_width: float = 8.0, # In PDF points
    min_region_height: float = 8.0, # In PDF points
    merge_iterations: int = 6,
    region_padding: float = 0.3, # Padding around merged regions for word capture (PDF points)
    line_break_threshold: float = 0.6, # Factor of word height to detect line breaks
    filter_numeric_blocks: bool = True, # Skip blocks containing only digits
    handle_unassigned_words: bool = False, # Option to process words not in any region
    visual_proof: bool = False
) -> List[Dict[str, Any]]:
    """
    Extracts text from a PDF page by detecting text regions using OpenCV
    on a rendered image, merging regions, and then extracting PyMuPDF text
    within those regions.

    Args:
        page: The pymupdf.Page object to process.
        zoom_factor: Factor to render the page image (higher zoom = more detail, slower).
        adaptive_thresh_block_size: Block size for OpenCV adaptive thresholding.
        adaptive_thresh_c: Constant subtracted in adaptive thresholding.
        morph_kernel_size: Kernel size for morphological operations (dilation, closing).
        dilate_iterations: Number of iterations for dilation.
        canny_thresh1: Lower threshold for Canny edge detection.
        canny_thresh2: Upper threshold for Canny edge detection.
        min_region_width: Minimum width of a valid detected region (in PDF points).
        min_region_height: Minimum height of a valid detected region (in PDF points).
        merge_iterations: Iterations for the (abstracted) merge_text_regions function.
        region_padding: Padding added around merged regions before capturing words.
        line_break_threshold: Factor of word height used in heuristic line break detection.
        filter_numeric_blocks: If True, blocks containing only digits are discarded.
        handle_unassigned_words: If True, attempt to process words not captured (NYI).
        visual_proof: If True, display intermediate processing images using matplotlib.

    Returns:
        A list of dictionaries, where each dictionary represents a detected
        text block and contains:
        - "occupy_space": The pymupdf.Rect of the merged region.
        - "content": The extracted text content (str).
        - "text_height_median": The median height of words in the block (float).
        - "min_height": The minimum height of words in the block (float).
        - "page_num": The page number (0-indexed).
    """
    print(f"Processing Page {page.number} via Image Regions...")

    # 1. Render Page to Image
    try:
        mat = pymupdf.Matrix(zoom_factor, zoom_factor)
        pix = page.get_pixmap(matrix=mat)
        img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.h, pix.w, pix.n)
    except Exception as e:
        print(f"  Error rendering page {page.number} to pixmap: {e}")
        return []

    # 2. OpenCV Image Processing to Find Contours
    # Convert to grayscale
    if img.shape[2] >= 3: # Check if color channels exist
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    elif img.shape[2] == 1:
         gray = img[:, :, 0]
    else: # Handle unexpected image format
         print(f"  Warning: Unexpected image format with shape {img.shape}. Trying first channel.")
         gray = img[:, :, 0] if len(img.shape) > 2 and img.shape[2] > 0 else img

    # Apply adaptive thresholding
    adaptive = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                     cv2.THRESH_BINARY_INV, adaptive_thresh_block_size, adaptive_thresh_c)

    # Morphological operations to connect characters/words into text block shapes
    rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, morph_kernel_size)
    dilated = cv2.dilate(adaptive, rectKernel, iterations=dilate_iterations)
    closing = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, rectKernel) 

    # Canny edge detection (sometimes helps refine contours)
    edged = cv2.Canny(closing, canny_thresh1, canny_thresh2, apertureSize=3)

    # Find external contours
    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    print(f"  Found {len(contours)} initial contours.")

    # 3. Create Initial Regions from Contours
    initial_regions = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        # Convert back to PDF coordinates
        pdf_x0, pdf_y0 = x / zoom_factor, y / zoom_factor
        pdf_x1, pdf_y1 = (x + w) / zoom_factor, (y + h) / zoom_factor
        rect = pymupdf.Rect(pdf_x0, pdf_y0, pdf_x1, pdf_y1)

        if rect.width >= min_region_width / 2 and rect.height >= min_region_height / 2: # Looser pre-filter
             initial_regions.append(rect)

    print(f"  Created {len(initial_regions)} initial regions after pre-filtering.")

    # 4. Merge Overlapping/Nearby Regions (Abstracted Call)
    merged_regions = merge_text_regions(initial_regions, iterations=merge_iterations)

    # Filter out regions that are too small after merging
    final_regions = [r for r in merged_regions
                     if r.width >= min_region_width and r.height >= min_region_height]
    print(f"  {len(final_regions)} regions remaining after merging and size filtering.")

    # Sort final regions by reading order (top-to-bottom, left-to-right)
    final_regions.sort(key=lambda r: (r.y0, r.x0))

    # 5. Extract Text Words using PyMuPDF
    words_data = page.get_text("words") # Format: (x0, y0, x1, y1, word, block_no, line_no, word_no)
    if not words_data:
        print(f"  No text words found on page {page.number} by PyMuPDF.")
        return []

    # Prepare word items with rectangles and assignment status
    word_items = [{
        "rect": pymupdf.Rect(w[:4]),
        "text": w[4],
        "baseline": w[1], 
        "height": pymupdf.Rect(w[:4]).height,
        "assigned": False
    } for w in words_data]

    # 6. Assign Words to Regions and Reconstruct Text Blocks
    extracted_blocks = []
    for region_rect in final_regions:
        # print(f"  Processing region: {region_rect}")
        # Define the padded capture area for the current region
        padded_rect = pymupdf.Rect(region_rect.x0 - region_padding,
                                   region_rect.y0 - region_padding,
                                   region_rect.x1 + region_padding,
                                   region_rect.y1 + region_padding)
        
        words_in_region_indices = []
        median_finder = MedianFinder() 
        # Find unassigned words that intersect the padded region
        for idx, item in enumerate(word_items):
            if not item["assigned"] and item["rect"].intersects(padded_rect):
                words_in_region_indices.append(idx)
                median_finder.addNum(item["height"])
                item["assigned"] = True 
                
        if not words_in_region_indices:
            # print(padded_rect)
            continue 
        current_block_words = [word_items[i] for i in words_in_region_indices]
        current_block_words.sort(key=lambda w: (w["rect"].y0, w["rect"].x0))

        
        block_text = ""
        min_height_in_block = float('inf')
        prev_baseline = current_block_words[0]["baseline"] if current_block_words else 0

        for word_item in current_block_words:
            current_baseline = word_item["baseline"]
            current_height = word_item["height"]
            if current_height > 0: 
                 min_height_in_block = min(min_height_in_block, current_height)
            # Heuristic: Check vertical distance between baselines relative to current word height
            # Use a small tolerance (e.g., 1 PDF point) for exact same baseline match
            vertical_distance = abs(prev_baseline - current_baseline)
            is_same_line = vertical_distance < current_height*0.75 or (current_height > 0 and vertical_distance <= current_height * line_break_threshold)

            separator = " " if is_same_line else "\n"
            if block_text: 
                block_text += separator
            block_text += word_item["text"]
            prev_baseline = current_baseline 

        content = block_text.strip()
        if not content: continue 

        if filter_numeric_blocks and content.isdigit():
            print(f"  Skipping numeric block: '{content}'")
            continue

        # Store the extracted block information
        extracted_blocks.append({
            "occupy_space": region_rect, 
            "content": content,
            "text_height_median": median_finder.findMedian(),
            "min_height": min_height_in_block if min_height_in_block != float('inf') else 0,
            "page_num": page.number
        })

    print(f"  Extracted {len(extracted_blocks)} text blocks.")

    # 7.  Handle Unassigned Words (Not Yet Implemented)
    unassigned_word_count = sum(1 for item in word_items if not item["assigned"])
    # print(*[(item["rect"], item["text"]) for item in word_items if not item["assigned"]], sep="\n")
    if unassigned_word_count > 0:
        print(f"  Warning: {unassigned_word_count} words were not assigned to any detected region.")
        if handle_unassigned_words:
            # --- Add logic here to potentially group/process unassigned words ---
            # Could involve creating default blocks based on proximity, line numbers etc.
            print("(Handling unassigned words not implemented)")
            pass
        # --- End NYI section ---


    # 8. Visual Proof (Optional)
    if visual_proof:
        try:
            viz_img = img.copy()
            # Draw final merged regions used for text extraction
            for item in extracted_blocks:
                rect = item["occupy_space"]
                # rect = item
                # print(item)
                # Convert PDF rect back to image coordinates for drawing
                x0, y0 = int(rect.x0 * zoom_factor), int(rect.y0 * zoom_factor)
                x1, y1 = int(rect.x1 * zoom_factor), int(rect.y1 * zoom_factor)
                cv2.rectangle(viz_img, (x0, y0), (x1, y1), (0, 255, 0), 2) # Green boxes

            plt.figure(figsize=(15, 15))

            # Show edge/contour image
            plt.subplot(1, 2, 1)
            plt.imshow(edged, cmap='gray')
            plt.title(f"Page {page.number} - Edges")
            plt.axis('off')

            # Show original image with detected regions overlaid
            plt.subplot(1, 2, 2)
            # Convert viz_img from BGR (OpenCV default) to RGB (matplotlib default) if necessary
            if viz_img.shape[2] == 3:
                viz_img_rgb = cv2.cvtColor(viz_img, cv2.COLOR_BGR2RGB)
                plt.imshow(viz_img_rgb)
            else: # Grayscale
                 plt.imshow(viz_img, cmap='gray')

            plt.title(f"Page {page.number} - Detected Text Regions ({len(extracted_blocks)} blocks)")
            plt.axis('off')
            plt.tight_layout()
            plt.show()
        except Exception as e:
            print(f"  Error generating visual proof: {e}")


    return extracted_blocks

In [8]:
def extract_text(pdf_path:str)-> tuple[List[dict], str]:
    """PyMuPDF-based function to extract text with bounding boxes from a PDF file."""
    try:
        doc = pymupdf.open(pdf_path, filetype="pdf")
        prev_block = None
        all_blocks = []
        for page_num, page in enumerate(doc):
            words = page.get_text("words")
            page_block = []
            # Take threshold based on page_width and page_height
            WIDTH_threshold = (0.02 if page.rect.width > page.rect.height else 0.0092625) * page.rect.width
            HEIGHT_threshold = (0.01 if page.rect.width > page.rect.height else 0.05) * page.rect.height
            for curr_word in words:
                # Each block is (x0, y0, x1, y1, text, block_no, block_type)
                x0, y0, x1, y1, text, block_no, line_no, block_type = curr_word
                text_height = y1 - y0
                if not text.strip():
                    continue
                is_mergable = False
                
                """
                Check if the current block is close to the previous block.
                The conditions are:
                (1. The x-coordinates of the current block are within WIDTH_threshold of the previous block.
                2. The y-coordinates of the current block are within HEIGHT_threshold of the previous block.
                (3. The current block is not completely to the left of the previous block.
                4. The y-coordinates of the current block are within 4 pixels of the previous block.
                5. The current block is not completely to the right of the previous block.
                """
                if prev_block and abs(text_height - prev_block[-1]) <= 0 and \
                    (\
                        (abs(x0 - prev_block[2]) <= WIDTH_threshold and (y0 -prev_block[3]) <= HEIGHT_threshold) \
                    ):
                    prev_block[2] = max(prev_block[2], x1) 
                    prev_block[3] = max(prev_block[3], y1) 
                    prev_block[4] += " " + text.strip()
                    is_mergable = True

                if is_mergable and page_block:
                    page_block.pop()
                    page_block.append(
                        {
                            "page": page_num,
                            "bbox": (prev_block[0], prev_block[1], prev_block[2], prev_block[3]),
                            "text": prev_block[4],
                            "block_no": block_no,
                            "block_type": block_type,
                            "text_height": prev_block[-1]
                        }
                    )    
                    prev_block = [prev_block[0], prev_block[1], prev_block[2], prev_block[3], prev_block[4], block_no, block_type, prev_block[-1]]
                else:
                    page_block.append({
                        "page": page_num,
                        "bbox": (x0, y0, x1, y1),
                        "text": text.strip(),
                        "block_no": block_no,
                        "block_type": block_type,
                        "text_height": text_height
                    })
                    # Update the previous block
                    prev_block = [x0, y0, x1, y1, text.strip(), block_no, block_type, text_height]
            # merged_blocks = process_text_blocks(page_block, eps=25, min_samples=1)
            all_blocks.extend(page_block)
        doc.close()
        # Remove empty blocks
        if not all_blocks:
            raise Exception("PyMuPDF Failed or No text found in the PDF.")
        return (all_blocks, 'pyMuPDF')
    except Exception as e:
        print("The Error is", e.with_traceback())
        # return extract_text_with_Mistral_OCR(pdf_path)



In [9]:
import time
DUMMY_NODE_HEIGHT = 1000

def extract_and_prepare_elements(pdf_path):
    """
    Opens PDF, extracts text elements using the provided function,
    ensures required keys are present, sorts by reading order per page,
    and returns a single list of all elements.
    """
    try:
        doc = pymupdf.open(pdf_path)
    except Exception as e:
        print(f"Error opening PDF {pdf_path}: {e}")
        return []

    all_elements = []
    num_pages = len(doc)
    print(f"Processing {num_pages} pages from {os.path.basename(pdf_path)}...")

    for i in range(num_pages):
        try:
            # Call the external extraction function (abstracted)
            page_elements = extract_page_text_via_image_regions(page=doc[i], visual_proof=False)

            # Validate and potentially enrich elements
            valid_page_elements = []
            for element in page_elements:
                 # Ensure essential keys exist (add defaults or skip if necessary)
                if not all(k in element for k in ["content", "occupy_space", "text_height_median"]):
                     print(f"    Warning: Skipping element on page {i+1} due to missing keys: {element.get('content', '[No Content]')[:50]}...")
                     continue
                element['page_num'] = i # Ensure correct page number is set
                valid_page_elements.append(element)


            # Sort elements on the current page by position (reading order)
            valid_page_elements.sort(key=lambda x: (x["occupy_space"].y0, x["occupy_space"].x0))
            all_elements.extend(valid_page_elements)

        except Exception as e:
            print(f"Error processing page {i + 1}: {e}")
            # Decide whether to continue or stop on page error
            continue

    print(f"Finished extraction. Total elements: {len(all_elements)}")
    doc.close() # Ensure the document is closed
    return all_elements


def identify_and_filter_watermarks(elements):
    """
    Identifies elements with identical stripped content appearing more than once
    and filters them out.
    """
    if not elements:
        return [], set()

    print("Identifying potential watermarks...")
    # Count occurrences of each unique stripped text content
    text_counts = collections.Counter(
        (el["content"].strip(), el.get("text_height_median", 0)) for el in elements if el.get("content")
    )
    # TODO: Factor in median text height
    # Identify texts that appear more than once as potential watermarks
    watermark_candidates = {(text[0], text[1]) for text, count in text_counts.items() if count > 1 and text} 

    if watermark_candidates:
        print(f"  Identified potential watermarks ({len(watermark_candidates)} unique texts): {watermark_candidates}")
        # Filter out elements whose stripped content is in the watermark set
        filtered_elements = [
            el for el in elements
            if el.get("content") and (el["content"].strip(), el["text_height_median"]) not in watermark_candidates
        ]
        print(f"  Removed {len(elements) - len(filtered_elements)} watermark instances. Elements remaining: {len(filtered_elements)}")
        return filtered_elements, watermark_candidates
    else:
        print("  No potential watermarks found.")
        return elements, set()


def calculate_child_spans(elements):
    """
    Calculates the 'span' for each element, representing how many subsequent,
    smaller-height elements fall under it before encountering an element of
    equal or greater height. This is used to determine parent-child relationships.

    Uses a stack-based approach to find the "next greater or equal element" boundary.
    """
    if not elements:
        return []

    print("Calculating hierarchy markers (child spans) based on text height...")
    # Add a dummy element at the end with a very large height.
    # This ensures all elements in the stack are processed correctly without needing extra end-of-loop logic.
    try:
        dummy_element = elements[-1].copy() if elements else {} # Handle empty list case
        dummy_element["text_height_median"] = DUMMY_NODE_HEIGHT
    except KeyError:
         print("Error: Last element missing 'text_height_median'. Cannot create dummy node reliably.")
         # Fallback or raise error - using arbitrary large height if copy fails but list not empty
         dummy_element = {"text_height_median": DUMMY_NODE_HEIGHT} if elements else {}


    processing_list = elements + [dummy_element]
    num_processing = len(processing_list)
    # Stores how many elements immediately following element `i` are 'under' it (smaller height).
    child_span_counts = [0] * num_processing
    stack = [] # Stores indices of elements waiting for their scope boundary (next >= element)

    for idx, current_element in enumerate(processing_list):
        current_height = current_element.get("text_height_median", 0) # Default height if missing

        # While stack is not empty AND current element's height is >= stack top element's height:
        # The current element marks the end of the scope for the element at the top of the stack.
        while stack and current_height >= processing_list[stack[-1]].get("text_height_median", 0):
            parent_index = stack.pop()
            # Calculate the span: number of elements between parent and current element
            child_span_counts[parent_index] = idx - parent_index - 1

        # Push the index of the current element onto the stack
        stack.append(idx)

    print("Finished calculating child spans.")
    return child_span_counts[:-1]


def build_hierarchy(elements, child_span_counts):
    """
    Builds a tree structure using Node objects based on the elements and their
    calculated child spans. Identifies root nodes (potential chapters/sections).
    """
    if not elements or len(elements) != len(child_span_counts):
        print("Warning: Mismatch between elements and child spans, or list is empty. Cannot build hierarchy.")
        return [], [] 

    print("Building hierarchy tree...")
    nodes = [Node(el) for el in elements]
    num_nodes = len(nodes)

    for i in range(num_nodes):
        span = child_span_counts[i]
        if span > 0:
            # This node `i` is a parent. Add its direct children.
            current_child_idx = i + 1
            while current_child_idx < min(i + 1 + span, num_nodes):
                # Add node at `current_child_idx` as a child of node `i`
                nodes[i].add_child(nodes[current_child_idx])
                grandchild_span = child_span_counts[current_child_idx]
                current_child_idx += (grandchild_span + 1) # Move past the child and its descendants

    # Identify root nodes: typically nodes that have children (parents).
    root_nodes = [node for i, node in enumerate(nodes) if child_span_counts[i] > 0]

    # Fallback: If no nodes have children (e.g., flat structure or all same height),
    # treat all original nodes as roots (potentially filtered later if needed).
    if not root_nodes and nodes:
        print("  Warning: No parent nodes identified based on height/span. Assuming flat structure or single level.")
        # Decide strategy: either all nodes are roots, or maybe just the first?
        # Original code implies all nodes become roots in this case if `sum(count_till_match_size) == 0`.
        # Let's check the span sum for that condition.
        if sum(child_span_counts) == 0:
             root_nodes = nodes
        else:
             # This case (nodes exist, spans exist, but no nodes selected as roots) seems unlikely with the logic.
             # If it happens, maybe select the first node or log an error.
             print("  Ambiguous root node condition. Defaulting to all nodes as roots.")
             root_nodes = nodes

    elif not nodes:
        print("  No text elements to form a hierarchy.")
        root_nodes = []
    else:
        print(f"  Identified {len(root_nodes)} potential root nodes (chapters/sections).")

    return nodes, root_nodes # Return all nodes and the identified roots


def generate_json_structure(root_nodes):
    """
    Generates a nested list of dictionaries (JSON structure) from the root nodes
    using Breadth-First Search (BFS) to traverse the hierarchy.
    """
    if not root_nodes:
        return []

    print("Generating JSON output...")
    output_json_list = []
    # Keep track of nodes already added to the JSON globally to handle
    # cases where the root detection might be imperfect or graphs are disconnected.
    visited_nodes = set()

    def bfs_to_dict(start_node):
        """Performs BFS from a node and builds the nested dictionary for its branch."""
        # Check if this node was already processed as part of another root's tree
        if start_node in visited_nodes:
            return None # Don't process this branch again

        # Use a queue for BFS: stores tuples of (node_to_process, dict_to_populate_for_that_node)
        # Initialize with the starting node and the dictionary that will represent it
        root_dict = {}
        queue = collections.deque([(start_node, root_dict)])
        # Keep track of nodes visited *within this specific BFS call* to prevent cycles within a component
        local_visited = {start_node}

        while queue:
            current_node, current_data_dict = queue.popleft()

            # Mark node as globally visited *after* processing its data
            visited_nodes.add(current_node)

            # Populate the dictionary for the current node
            content_text = current_node.content.get("content", "")
            current_data_dict.update({
                "name": content_text.strip(),
                "children": []
                })

            # Prepare children list and add valid children to the queue
            child_list = current_data_dict["children"]
            for child_node in current_node.children:
                # Process child only if not visited globally or within this BFS call
                if child_node not in visited_nodes and child_node not in local_visited:
                    local_visited.add(child_node)
                    child_data = {} # Create a new dict for the child
                    child_list.append(child_data) # Add the placeholder dict to parent's children list
                    queue.append((child_node, child_data)) # Add child node and its dict to the queue

        return root_dict # Return the populated dictionary for the starting node's branch

    # Iterate through the identified root nodes
    for root in root_nodes:
        branch_dict = bfs_to_dict(root)
        if branch_dict: # Only add if the BFS generated a structure (i.e., node wasn't already visited)
            output_json_list.append(branch_dict)

    print(f"Finished generating JSON structure with {len(output_json_list)} top-level items.")
    return output_json_list


def save_to_json(data, output_path):
    """Saves the provided data structure to a JSON file."""
    print(f"Saving structured JSON to: {output_path}")
    try:
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        print("Successfully saved JSON.")
    except IOError as e:
        print(f"Error writing JSON file to {output_path}: {e}")
    except TypeError as e:
        print(f"Error serializing data to JSON (check data structure): {e}")


# --- Main Processing Function ---

def process_pdf_to_structured_json(input_pdf_path, output_json_path=None):
    """
    Main orchestrator function:
    1. Checks input path.
    2. Extracts and prepares text elements.
    3. Identifies and filters watermarks.
    4. Sorts elements globally for hierarchy processing.
    5. Calculates child spans based on height.
    6. Builds the node hierarchy.
    7. Generates the final JSON structure.
    8. Saves the JSON file.
    """
    start_time = time.time()
    print(f"\n--- Starting processing for: {input_pdf_path} ---")

    if not os.path.exists(input_pdf_path):
        print(f"Error: Input PDF not found at {input_pdf_path}")
        return

    if output_json_path is None:
        output_json_path = os.path.splitext(input_pdf_path)[0] + "_structured.json"

    # 1. Extract and Prepare Elements (Abstracted Call)
    #    CRITICAL: Requires extract_text_with_ocr_approach to return dicts with
    #    'content', 'occupy_space' (Rect-like), 'text_height_median', 'page_num'.
    elements = extract_and_prepare_elements(input_pdf_path)
    if not elements:
        print("No text elements extracted or processed. Aborting.")
        return

    # 2. Identify and Filter Watermarks
    elements, _ = identify_and_filter_watermarks(elements) # Watermark list ignored for now
    if not elements:
        print("No content remaining after watermark removal. Aborting.")
        return

    # 3. Sort All Elements Globally for Hierarchy Building
    #    Sort primarily by page, then y-position (top-to-bottom), then x-position (left-to-right).
    print("Sorting all elements for hierarchical processing...")
    elements.sort(key=lambda x: (
        x.get("page_num", 0),
        x.get("occupy_space").y0 if x.get("occupy_space") else 0,
        x.get("occupy_space").x0 if x.get("occupy_space") else 0
    ))
    print("Sorting complete.")

    # 4. Calculate Hierarchy Markers (Child Spans)
    child_span_counts = calculate_child_spans(elements)

    # 5. Build Hierarchy Tree
    all_nodes, root_nodes = build_hierarchy(elements, child_span_counts)
    if not root_nodes:
         print("No hierarchical structure could be determined. Aborting JSON generation.")
         # Optionally, could still output a flat list here if desired.
         return

    # 6. Generate JSON Output from Tree Roots
    output_data = generate_json_structure(root_nodes)

    # 7. Save the JSON Output
    save_to_json(output_data, output_json_path)

    end_time = time.time()
    print(f"--- Processing finished in {end_time - start_time:.2f} seconds ---")


# --- Example Usage ---
if __name__ == "__main__":
    pdf_to_process =r"D:\DATA300\AudioBookSum\pdf\Didi Huberman.pdf"
    
    # Check if the placeholder path is still set
    if pdf_to_process == "path/to/your/document.pdf":
        print("\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print("!!! Please update the 'pdf_to_process' variable in the   !!!")
        print("!!! '__main__' block of the script before running.        !!!")
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n")
    elif not os.path.exists(pdf_to_process):
         print(f"\nWarning: The specified PDF file does not exist: {pdf_to_process}")
         print("Script will not run processing.")
    else:
        # Run the main processing pipeline
        process_pdf_to_structured_json(pdf_to_process)


--- Starting processing for: D:\DATA300\AudioBookSum\pdf\Didi Huberman.pdf ---
Processing 59 pages from Didi Huberman.pdf...
Processing Page 0 via Image Regions...
  Found 14 initial contours.
  Created 14 initial regions after pre-filtering.
  3 regions remaining after merging and size filtering.
  No text words found on page 0 by PyMuPDF.
Processing Page 1 via Image Regions...
  Found 6 initial contours.
  Created 6 initial regions after pre-filtering.
  5 regions remaining after merging and size filtering.
  Extracted 5 text blocks.
Processing Page 2 via Image Regions...
  Found 9 initial contours.
  Created 9 initial regions after pre-filtering.
  9 regions remaining after merging and size filtering.
  Skipping numeric block: '2002029382'
  Extracted 8 text blocks.
Processing Page 3 via Image Regions...
  Found 23 initial contours.
  Created 23 initial regions after pre-filtering.
  19 regions remaining after merging and size filtering.
  Skipping numeric block: '1'
  Skipping num

In [10]:
%%time
input_pdf_path = r"D:\DATA300\AudioBookSum\pdf\Didi Huberman.pdf"
words, extract_method = extract_text(input_pdf_path)

CPU times: total: 203 ms
Wall time: 219 ms


In [11]:
%%time
input_pdf_path = r"D:\DATA300\AudioBookSum\pdf\Didi Huberman.pdf"
doc = pymupdf.open(input_pdf_path)
text = ""
median_collection = []
content = []

for i in range(len(doc)):
    extracted_pages = sorted(extract_text_with_ocr_approach(input_pdf_path, page_num =i, visual_proof=False), \
        key = lambda x: [x["occupy_space"].y0, x["occupy_space"].x0])
    content.extend(extracted_pages)

CPU times: total: 3.73 s
Wall time: 3.54 s


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_text(text_blocks, chunk_size=1000, chunk_overlap=100):
    """
    Chunk text blocks into smaller pieces while preserving page and bounding box information.
    
    Args:
        text_blocks (list): List of dictionaries containing text with metadata (page, bbox, etc.)
        chunk_size (int): Maximum size of each chunk in characters
        chunk_overlap (int): Number of overlapping characters between chunks
        
    Returns:
        list: List of dictionaries containing chunks with preserved metadata
    """
    chunked_data = []  
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    
    # First combine all text blocks into a single document string for chunking
    combined_text = ""
    metadata_map = {}  # Map character positions to original metadata
    current_position = 0
    
    # Process each text block and map character positions to metadata
    for idx, block in enumerate(text_blocks):
        # Get text and metadata
        text = block.get("content", "")
        if not text:
            continue
            
        block_length = len(text)
        page = block.get("page_num", 0)
        bbox = block.get("occupy_space", None)
        text_height = block.get("text_height_median", None)
        
        # Map each character position in the combined text to its source metadata
        for i in range(current_position, current_position + block_length):
            metadata_map[i] = {
                "page": page,
                "bbox": bbox,
                "text_height": text_height,
                "original_block_index": idx
            }
        
        # Add text to combined string with a marker between blocks
        combined_text += text + " [BLOCK_BREAK] "
        current_position += block_length + len(" [BLOCK_BREAK] ")
    
    # Split the combined text into chunks
    chunks = text_splitter.split_text(combined_text)
    
    # For each chunk, determine the predominant page and bounding box
    chunk_start = 0
    for i, chunk in enumerate(chunks):
        # Find the start position of this chunk in the combined text
        if i == 0:
            start_pos = 0
        else:
            # Look for chunk in the vicinity of where we expect it
            search_start = max(0, chunk_start - chunk_overlap)
            search_text = combined_text[search_start:search_start + len(chunk) + 100]
            rel_pos = search_text.find(chunk[:50])  # Use prefix to locate chunk
            if rel_pos != -1:
                start_pos = search_start + rel_pos
            else:
                # Fallback if exact match not found
                start_pos = chunk_start
        
        # Update for next iteration
        chunk_start = start_pos + len(chunk) - chunk_overlap
            
       
        page_counts = {}
        bbox_by_page = {}
        
        for j in range(start_pos, min(start_pos + len(chunk), len(combined_text))):
            if j in metadata_map:
                page = metadata_map[j]["page"]
                if page not in page_counts:
                    page_counts[page] = 0
                    bbox_by_page[page] = []
                    
                page_counts[page] += 1
                if metadata_map[j]["bbox"] and metadata_map[j]["bbox"] not in bbox_by_page[page]:
                    bbox_by_page[page].append(metadata_map[j]["bbox"])
        
        # Find predominant page
        predominant_page = max(page_counts.items(), key=lambda x: x[1])[0] if page_counts else 0
        
        # Create enhanced chunk with metadata
        enhanced_chunk = {
            "text": chunk.replace(" [BLOCK_BREAK] ", "\n\n"),
            "page": predominant_page,
            "bbox_list": bbox_by_page.get(predominant_page, []),
            "start_position": start_pos,
            "chunk_index": i
        }
        
        chunked_data.append(enhanced_chunk)
    
    return chunked_data

In [13]:
def identify_interesting_points(chunked_data, llm, text_blocks, file_name, threshold=5):
    """
    Identify important points in text using LLM, adapted to work with chunk_text output.
    
    Args:
        chunked_data (list): List of dictionaries returned by chunk_text function
        llm: LLM model for generating interesting points
        text_blocks (list): Original text blocks with metadata
        file_name (str): Path to cache file
        threshold (int): Minimum word count for segments to be considered
        
    Returns:
        list: List of interesting sections with metadata and page annotations
    """
    interesting_sections = []
    
    # Return early if chunked_data is empty
    if not chunked_data:
        print("No content identified in the document")
        return interesting_sections
    
    # Create block index for faster lookup - map page numbers to blocks on that page
    page_to_blocks = defaultdict(list)
    for block in text_blocks:
        # Pre-normalize block text once
        content = block.get("content", "")
        if content:
            block["norm_text"] = ' '.join(content.split())
            page_to_blocks[block.get("page_num", 0)].append(block)
    
    # Function to find matching block for a segment with improved fuzzy matching
    def find_matching_block(segment, page_hint=None):
        print(f"Finding matching block for: {segment[:50]} ...") 
        print(f"Page hint: {page_hint}")
        norm_segment = ' '.join(segment.split())
        if not norm_segment:
            return None, False
            
        # First try exact page if provided
        if page_hint is not None:
            for block in page_to_blocks.get(page_hint, []):
                if norm_segment in block["norm_text"]:
                    return block, True
        
        # Try exact matching across all pages
        for page, page_blocks in page_to_blocks.items():
            for block in page_blocks:
                if norm_segment in block["norm_text"]:
                    return block, True
        
        # Try finding any substantial overlap
        best_match = None
        highest_similarity = 0.6  # Lowered threshold for better recall
        best_overlap_len = 0
        
        # First prioritize matches by page if available
        if page_hint is not None:
            for block in page_to_blocks.get(page_hint, []):
                # Try SequenceMatcher similarity
                similarity = difflib.SequenceMatcher(None, norm_segment, block["norm_text"]).ratio()
                if similarity > highest_similarity:
                    highest_similarity = similarity
                    best_match = block
                
                # Try finding common substring
                common_words = set(norm_segment.split()) & set(block["norm_text"].split())
                if len(common_words) > best_overlap_len:
                    best_overlap_len = len(common_words)
                    if not best_match:  # Only replace if we don't have a high similarity match
                        best_match = block
        
        # If no good match on the hinted page, try all pages
        if not best_match or highest_similarity < 0.7:
            for page, page_blocks in page_to_blocks.items():
                for block in page_blocks:
                    similarity = difflib.SequenceMatcher(None, norm_segment, block["norm_text"]).ratio()
                    if similarity > highest_similarity:
                        highest_similarity = similarity
                        best_match = block
                    
                    # Try finding common substring as backup approach
                    common_words = set(norm_segment.split()) & set(block["norm_text"].split())
                    overlap_ratio = len(common_words) / max(1, len(norm_segment.split()))
                    if overlap_ratio > 0.5 and len(common_words) > best_overlap_len:
                        best_overlap_len = len(common_words)
                        if highest_similarity < 0.7:  # Only use word overlap if similarity isn't high
                            best_match = block
        
        # If still no match, but we have a page hint, return any block from that page
        if not best_match and page_hint is not None and page_hint in page_to_blocks and page_to_blocks[page_hint]:
            return page_to_blocks[page_hint][0], False
            
        # If still no match, return any block from any page as last resort
        if not best_match and page_to_blocks:
            first_page = min(page_to_blocks.keys())
            if page_to_blocks[first_page]:
                return page_to_blocks[first_page][0], False
        
        return best_match, False
    
    # Function to extract page number from text with [Page X] annotation
    def extract_page_number(text):
        page_match = re.search(r'\[Page (\d+)\]', text)
        if page_match:
            return int(page_match.group(1)) - 1  # Convert to 0-indexed
        return None
    
    # Cache handling using structured JSON format
    if os.path.exists(file_name):
        try:
            print(f"Loading cached interesting points from {file_name}")
            # Try JSON format first (for files created by the optimized version)
            try:
                with open(file_name, "r", encoding="utf-8") as f:
                    cached_data = json.load(f)
                    
                for segment_data in cached_data.get("segments", []):
                    segment = segment_data.get("text", "")
                    page_hint = segment_data.get("page")
                    
                    if len(segment.split()) < threshold:
                        continue
                    
                    # Check for page number in the text itself
                    text_page = extract_page_number(segment)
                    if text_page is not None:
                        page_hint = text_page
                        # Remove the page annotation if it exists
                        segment_for_matching = re.sub(r'\[Page \d+\]\s*', '', segment).strip()
                    else:
                        segment_for_matching = segment
                    
                    matched_block, exact_match = find_matching_block(segment_for_matching, page_hint)
                    if matched_block:
                        # Add page annotation to the text
                        page_num = matched_block.get("page_num", 0)
                        page_annotation = f"[Page {page_num+1}]"
                        
                        # Keep original text but ensure page annotation is correct
                        if text_page is None:
                            clean_segment = segment_for_matching
                            display_text = f"{page_annotation} {clean_segment}"
                        elif text_page != page_num:
                            # Replace with correct page
                            clean_segment = re.sub(r'\[Page \d+\]\s*', '', segment).strip()
                            display_text = f"{page_annotation} {clean_segment}"
                        else:
                            display_text = segment  # Keep original if page is correct
                        
                        interesting_sections.append({
                            "page": page_num,
                            "text": display_text,
                            "category": "main",
                            "bbox": matched_block.get("occupy_space"),
                            "page_text": f"Page {page_num+1}"
                        })
                    else:
                        # Still keep the segment but with a default page
                        default_page = page_hint if page_hint is not None else 0
                        page_annotation = f"[Page {default_page+1}]"
                        
                        # Handle page annotation in text
                        if text_page is None:
                            display_text = f"{page_annotation} {segment}"
                        else:
                            # Keep original annotation
                            display_text = segment
                        
                        print(f"Using default page for unmatched segment: {segment[:50]}...")
                        interesting_sections.append({
                            "page": default_page,
                            "text": display_text,
                            "category": "main",
                            "bbox": None,
                            "page_text": f"Page {default_page+1}"
                        })
                
            except json.JSONDecodeError:
                # Fall back to the legacy format for backward compatibility
                with open(file_name, "r", encoding="utf-8") as f:
                    content = f.read()
                
                cached_segments = re.findall(r'```segment(?:\s*-\s*(\d+))?\s*(.*?)```', content, re.DOTALL)
                
                for page_num_str, segment in cached_segments:
                    segment = segment.strip()
                    if len(segment.split()) < threshold:
                        continue
                    
                    # Try to get page number from the text first
                    text_page = extract_page_number(segment)
                    if text_page is not None:
                        page_hint = text_page
                        # Remove the page annotation if it exists
                        segment_for_matching = re.sub(r'\[Page \d+\]\s*', '', segment).strip()
                    else:
                        page_hint = int(page_num_str) - 1 if page_num_str else None
                        segment_for_matching = segment
                    
                    matched_block, exact_match = find_matching_block(segment_for_matching, page_hint)
                    
                    if matched_block:
                        # Add page annotation to the text
                        page_num = matched_block.get("page_num", 0)
                        page_annotation = f"[Page {page_num+1}]"
                        
                        # Keep original text but ensure page annotation is correct
                        if text_page is None:
                            clean_segment = segment_for_matching
                            display_text = f"{page_annotation} {clean_segment}"
                        elif text_page != page_num:
                            # Replace with correct page
                            clean_segment = re.sub(r'\[Page \d+\]\s*', '', segment).strip()
                            display_text = f"{page_annotation} {clean_segment}"
                        else:
                            display_text = segment  # Keep original if page is correct
                        
                        interesting_sections.append({
                            "page": page_num,
                            "text": display_text,
                            "category": "main",
                            "bbox": matched_block.get("occupy_space"),
                            "page_text": f"Page {page_num+1}"
                        })
                    else:
                        # Still keep the segment but with a default page
                        default_page = page_hint if page_hint is not None else 0
                        page_annotation = f"[Page {default_page+1}]"
                        
                        if text_page is None:
                            display_text = f"{page_annotation} {segment}"
                        else:
                            # Keep original annotation
                            display_text = segment
                        
                        print(f"Using default page for unmatched segment: {segment[:50]}...")
                        interesting_sections.append({
                            "page": default_page,
                            "text": display_text,
                            "category": "main",
                            "bbox": None,
                            "page_text": f"Page {default_page+1}"
                        })
        
        except Exception as e:
            print(f"Error loading cached data: {e}. Generating new segments.")
            interesting_sections = []  # Reset and try generating new
    
    # Generate new segments if no cached results were loaded
    if not interesting_sections:
        try:
            print("Generating new interesting points with LLM...")
            
            prompt_template = PromptTemplate(
                input_variables=["text"],
                template="""
                You are a reasoning summarizer.
                Summarize the provided text and support your summary using different verbatim snippets from the original text.
                Remember:
                The reasoning section must ONLY contain verbatim text from the document.
                Every sentence in the reasoning must be supporting sentences in the summary section.
                Do not add any information that isn't directly from the document.
                IMPORTANT: Do not remove page annotations like [Page X] from your snippets.
                
                Format each important snippet as:
                ```segment - <page number>
                <exact text from the document including the [Page X] annotation>
                ```
                
                Below is text from the main content of a document in English:
                {text}
                """
            )
            
            # Concatenate text from all chunks
            all_text = ""
            for chunk in chunked_data:
                page_num = chunk["page"]
                all_text += f"[Page {page_num+1}] {chunk['text']}\n\n"
                
            prompt = prompt_template.format(text=all_text)
            
            response = llm.invoke(prompt)
            content = response.content
            print("LLM response received")
            # Extract segments
            segments = re.findall(r'```segment(?:\s*-\s*(\d+))?\s*(.*?)```', content, re.DOTALL)
            print(f"Extracted {len(segments)} segments from LLM response")
            
            # Save in structured JSON format for better caching
            cached_data = {"segments": []}
            
            for page_num_str, segment_text in segments:
                segment = segment_text.strip()
                if len(segment.split()) < threshold:
                    continue
                
                # Try to extract page number from the text content first
                text_page = extract_page_number(segment)
                if text_page is not None:
                    page_hint = text_page
                    print(f"Found page annotation in text: Page {page_hint+1}")
                    # Remove page annotation for matching but keep original text
                    segment_for_matching = re.sub(r'\[Page \d+\]\s*', '', segment).strip()
                else:
                    page_hint = int(page_num_str) - 1 if page_num_str else None
                    print(f"Using segment marker page: Page {page_hint+1 if page_hint is not None else 'unknown'}")
                    segment_for_matching = segment
                
                matched_block, exact_match = find_matching_block(segment_for_matching, page_hint)
                
                # Always save the segment, even if no match is found
                if matched_block:
                    page_num = matched_block.get("page_num", 0)
                    page_annotation = f"[Page {page_num+1}]"
                    
                    # Store original segment in cache
                    segment_data = {
                        "page": page_num,
                        "text": segment,
                    }
                    cached_data["segments"].append(segment_data)
                    
                    # For display, ensure we have the correct page number
                    if text_page is None:
                        # No existing page annotation, add one
                        display_text = f"{page_annotation} {segment_for_matching}"
                    elif text_page != page_num:
                        # Replace incorrect page annotation
                        clean_segment = re.sub(r'\[Page \d+\]\s*', '', segment).strip()
                        display_text = f"{page_annotation} {clean_segment}"
                    else:
                        display_text = segment  # Keep original if it had the correct page
                    
                    interesting_sections.append({
                        "page": page_num,
                        "text": display_text,
                        "category": "main",
                        "bbox": matched_block.get("occupy_space"),
                        "page_text": f"Page {page_num+1}"
                    })
                else:
                    # Still save the segment with the hinted page or default to page 0
                    default_page = page_hint if page_hint is not None else 0
                    
                    # Store the original segment in cache
                    segment_data = {
                        "page": default_page,
                        "text": segment,
                    }
                    cached_data["segments"].append(segment_data)
                    
                    # For display, use the page annotation from the text or add a default one
                    if text_page is None:
                        page_annotation = f"[Page {default_page+1}]"
                        display_text = f"{page_annotation} {segment}"
                    else:
                        display_text = segment  # Keep original with its annotation
                    
                    print(f"Using default page {default_page+1} for unmatched segment: {segment[:50]}...")
                    interesting_sections.append({
                        "page": default_page,
                        "text": display_text,
                        "category": "main",
                        "bbox": None,
                        "page_text": f"Page {default_page+1}"
                    })
            
            # Save results in structured format
            with open(file_name, "w", encoding="utf-8") as f:
                json.dump(cached_data, f, indent=2)
            
        except Exception as e:
            print(f"Error during interesting point extraction: {e}")
            import traceback
            traceback.print_exc()
    
    print(f"Found {len(interesting_sections)} interesting sections")
    return interesting_sections

In [14]:
def highlight_interesting_points(pdf_path, interesting_points, output_path):
    """Add highlights to the interesting points in the PDF with complete text block highlighting."""
    doc = pymupdf.open(pdf_path)
    
    # Using cyan highlight color for main content
    highlight_color = (0, 1, 1)  # RGB for cyan
    fail_count = 0
    success_count = 0
    
    for point in interesting_points:
        page_num = point["page"]
        page = doc[page_num]
        text = point["text"]
        if not text:
            print(f"Empty text for page {page_num}")
            continue
            
        # Extract content without page annotations for better matching
        clean_text = re.sub(r'\[Page \d+\]\s*', '', text).strip()
        if not clean_text:
            continue
            
        # Try exact match first
        text_instances = page.search_for(clean_text)
        
        found_matches = []
        if not text_instances:
            # Method 1: Try normalized text (remove extra whitespace)
            normalized_text = ' '.join(clean_text.split())
            text_instances = page.search_for(normalized_text)
            
            # Method 2: Try with key phrases and collect all matches to combine later
            if len(normalized_text.split()) > 10:
                # Extract significant phrases (5-8 words)
                words = normalized_text.split()
                for i in range(len(words) - 5):
                    phrase = ' '.join(words[i:i+min(8, len(words)-i)])
                    if len(phrase) > 15:  # Only phrases with enough content
                        phrase_instances = page.search_for(phrase)
                        if phrase_instances:
                            found_matches.extend(phrase_instances)
            
            # Method 3: Use key sentences and collect all matches
            if '.' in normalized_text:
                sentences = [s.strip() for s in normalized_text.split('.') if len(s.strip()) > 15]
                for sentence in sentences:
                    sentence_instances = page.search_for(sentence)
                    if sentence_instances:
                        found_matches.extend(sentence_instances)
        
        # If we found partial matches, combine their bounding boxes to highlight full area
        if found_matches and not text_instances:
            if len(found_matches) >= 2:
                # Sort rectangles by y-coordinate and then by x-coordinate
                found_matches.sort(key=lambda r: (r[1], r[0]))  # Sort by top-y then left-x
                
                # Combine all matches into one larger rectangle
                x0 = min(rect[0] for rect in found_matches)
                y0 = min(rect[1] for rect in found_matches)
                x1 = max(rect[2] for rect in found_matches)
                y1 = max(rect[3] for rect in found_matches)
                
                # Create a single combined rectangle that covers all matches
                combined_rect = (x0, y0, x1, y1)
                text_instances = [combined_rect]
            else:
                text_instances = found_matches
        
        # Highlight found instances or use bbox as fallback
        if text_instances:
            for inst in text_instances:
                highlight = page.add_highlight_annot(inst)
                highlight.set_colors(stroke=highlight_color)
                highlight.update()
            success_count += 1
        elif point.get("bbox_list") and isinstance(point["bbox_list"], list) and point["bbox_list"]:
            # If we have a list of bounding boxes, highlight all of them
            for bbox in point["bbox_list"]:
                if bbox:  # Make sure bbox is not None
                    r = page.add_highlight_annot(bbox)
                    r.set_colors(stroke=highlight_color)
                    r.update()
            success_count += 1
        elif point.get("bbox"):
            # Single bbox case
            r = page.add_highlight_annot(point["bbox"])
            r.set_colors(stroke=highlight_color)    
            r.update()
            success_count += 1
        else:
            # Last resort: try to find a text chunk on the page containing some keywords
            if len(clean_text.split()) > 3:
                key_words = [w for w in clean_text.split() if len(w) > 5][:5]
                for word in key_words:
                    word_instances = page.search_for(word)
                    if word_instances:
                        for inst in word_instances:
                            highlight = page.add_highlight_annot(inst)
                            highlight.set_colors(stroke=(1, 0.5, 0))  # Orange for partial matches
                            highlight.update()
                        success_count += 1
                        break
            else:
                fail_count += 1
    
    print(f"Successfully highlighted {success_count} segments")
    print(f"Failed to highlight {fail_count} segments")
    
    # Save the highlighted PDF
    doc.save(output_path)
    doc.close()

In [15]:
base_name = os.path.splitext(input_pdf_path)[0]
output = f"{base_name}_highlighted.pdf"
chunked_data = chunk_text(content)
# Initialize LLM
try:
    llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash', temperature=0.7)
except Exception as e:
    print(f"Error initializing Gemini LLM: {e}")
    print("Make sure you have set GOOGLE_API_KEY in your environment or .env file")
    exit(1)
words, extract_method = extract_text(input_pdf_path)
# print(extract_method)

print(f"Extracted {len(words)} text blocks")

# Classify text blocks
# classified_blocks = classify_text_blocks(words, extract_method)
# print("Classified text blocks")
# # Identify interesting points (main content only)
interesting_points = identify_interesting_points(chunked_data, llm, words, f"{base_name}_interesting_points.txt")  # Save to file
# load from memory
print(f"Identified {len(interesting_points)} interesting points in main content")

# # Highlight interesting points in the PDF
highlight_interesting_points(input_pdf_path, interesting_points, output)
print(f"Created highlighted PDF: {output}")

print("Processing complete!")

Extracted 1763 text blocks
Generating new interesting points with LLM...
LLM response received
Extracted 5 segments from LLM response
Using segment marker page: Page unknown
Finding matching block for: - <Page 22>
 It would seem that hystero-epilepsy e ...
Page hint: None
Using default page 1 for unmatched segment: - <Page 22>
 It would seem that hystero-epilepsy e...
Using segment marker page: Page unknown
Finding matching block for: - <Page 23>
 All this forms a whole whose parts fo ...
Page hint: None
Using default page 1 for unmatched segment: - <Page 23>
 All this forms a whole whose parts fo...
Using segment marker page: Page unknown
Finding matching block for: - <Page 41>
 Determining the facies appropriate to ...
Page hint: None
Using default page 1 for unmatched segment: - <Page 41>
 Determining the facies appropriate to...
Using segment marker page: Page unknown
Finding matching block for: - <Page 52>
 They were searching for the facies in ...
Page hint: None
Using default pa